In [166]:
using LinearAlgebra
using TensorKit
using KrylovKit
using JSON
using HDF5, JLD
cd("D:\\My Documents\\Code\\Julia_codes\\Tensor network\\IPEPS_TensorKit\\kagome\\SU2_PG")
#push!(LOAD_PATH, "D:\\My Documents\\Code\\Julia_codes\\Tensor network\\IPEPS_TensorKit\\kagome\\SU2_PG")
include("kagome_load_tensor.jl")
include("kagome_CTMRG.jl")
include("kagome_model.jl")
include("kagome_IPESS.jl")

D=6;
chi=5;
W=10;
N=6;
tol=1e-6;

Dtrun=W*W*chi*chi;

pow=Int((N-2)/2);

J1=1;
J2=0;
J3=0;
Jchi=0;
Jtrip=0;


In [167]:

parameters=Dict([("J1", J1), ("J2", J2), ("J3", J3), ("Jchi", Jchi), ("Jtrip", Jtrip)]);

A_set,B_set,A1_set,A2_set, A_set_occu,B_set_occu,A1_set_occu,A2_set_occu, S_label, Sz_label, virtual_particle, Va, Vb=construct_tensor(D);

filenm="LS_D_"*string(D)*"_chi_40.json"
json_dict=read_json_state(filenm);

bond_tensor,triangle_tensor=construct_su2_PG_IPESS(json_dict,A_set,B_set,A1_set,A2_set, A_set_occu,B_set_occu,A1_set_occu,A2_set_occu, S_label, Sz_label, virtual_particle, Va, Vb);

PEPS_tensor=bond_tensor;
@tensor PEPS_tensor[:] := bond_tensor[-1,1,-5]*bond_tensor[4,3,-6]*bond_tensor[-4,2,-7]*triangle_tensor[1,3,2]*triangle_tensor[4,-2,-3];
A_unfused=PEPS_tensor;

U_phy=unitary(fuse(space(PEPS_tensor, 5) ⊗ space(PEPS_tensor, 6) ⊗ space(PEPS_tensor, 7)), space(PEPS_tensor, 5) ⊗ space(PEPS_tensor, 6) ⊗ space(PEPS_tensor, 7));
@tensor A_fused[:] :=PEPS_tensor[-1,-2,-3,-4,1,2,3]*U_phy[-5,1,2,3];

init=Dict([("CTM", []), ("init_type", "PBC")]);
conv_check="singular_value";
@time CTM, AA_fused, U_L,U_D,U_R,U_U=CTMRG(A_fused,chi,conv_check,tol,init);

@time E_up, E_down=evaluate_ob(parameters, U_phy, A_unfused, A_fused, AA_fused, CTM, "E_triangle");
@time E_up_12, E_up_31, E_up_23, E_down_12, E_down_31, E_down_23=evaluate_ob(parameters, U_phy, A_unfused, A_fused, AA_fused, CTM, "E_bond");


"initialize CTM"

"start CTM iterations:"

  0.342428 seconds (678.42 k allocations: 69.262 MiB, 24.19% compilation time)
CTMRG iteration: 1, CTMRG err: 1.1502521239562251
CTMRG iteration: 2, CTMRG err: 0.010832246339709029
CTMRG iteration: 3, CTMRG err: 0.0009922670013661206
CTMRG iteration: 4, CTMRG err: 0.0006527577635839776
CTMRG iteration: 5, CTMRG err: 0.00018734167219991438
CTMRG iteration: 6, CTMRG err: 7.165737033032471e-5
CTMRG iteration: 7, CTMRG err: 3.049804770288711e-5
CTMRG iteration: 8, CTMRG err: 1.3703769502104723e-5
CTMRG iteration: 9, CTMRG err: 6.16076268211696e-6
CTMRG iteration: 10, CTMRG err: 2.786436377688606e-6
CTMRG iteration: 11, CTMRG err: 1.2619026413807077e-6
CTMRG iteration: 12, CTMRG err: 5.719815936999773e-7
  2.851950 seconds (7.71 M allocations: 712.063 MiB, 5.43% gc time, 35.13% compilation time)
  2.747652 seconds (5.36 M allocations: 561.303 MiB, 7.69% gc time, 55.08% compilation time)
  1.061366 seconds (3.73 M allocations: 477.091 MiB, 10.34% gc time, 20.14% compilation time)


In [168]:
display((E_up+E_down)/3)
Tleft=CTM["Tset"][4];
Tright=CTM["Tset"][2];

@tensor O1[:]:=Tleft[-3,1,-1]*U_L[1,-2,-4];
@tensor O2[:]:=Tright[-1,1,-3]*U_R[-4,-2,1];


-0.4269686722695194 + 2.3536858397074248e-17im

In [169]:
function HV_L_tensor(vl,A,mpo)
    if mpo==[]
        if numind(vl)==3
            @tensor vl[:]:=vl[-1,3,1]*A'[3,-2,2]*A[1,-3,2];
        elseif numind(vl)==2
            @tensor vl[:]:=vl[3,1]*A'[3,-2,2]*A[1,-3,2];
        end
    else
        if numind(vl)==5
            @tensor vl[:]:=vl[-1,7,5,3,1]*A'[7,-2,6]*mpo'[5,6,-3,4]*mpo[3,2,-4,4]*A[1,-5,2];
        elseif numind(vl)==4
            @tensor vl[:]:=vl[7,5,3,1]*A'[7,-1,6]*mpo'[5,6,-2,4]*mpo[3,2,-3,4]*A[1,-4,2];
        end
    end
    return vl
end

function HV_R_tensor(vr,A,mpo)
    if mpo==[]
        if numind(vr)==3
            @tensor vr[:]:=A'[-1,1,2]*A[-2,3,2]*vr[1,3,-3];
        elseif numind(vr)==2
            @tensor vr[:]:=A'[-1,1,2]*A[-2,3,2]*vr[1,3];
        end
    else
        if numind(vr)==5
            @tensor vr[:]:=A'[-1,7,6]*mpo'[-2,6,5,4]*mpo[-3,2,3,4]*A[-4,1,2]*vr[7,5,3,1,-5];
        elseif numind(vr)==4
            @tensor vr[:]:=A'[-1,7,6]*mpo'[-2,6,5,4]*mpo[-3,2,3,4]*A[-4,1,2]*vr[7,5,3,1];
        end
    end
    return vr
end

function left_eigenvector(A,mpo,type)

    if type=="A"
        HVfun1(x)=HV_L_tensor(x,A,[]);
        vl_init = permute(TensorMap(randn, space(A,1), space(A,1)), (1,2,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        eu,ev=eigsolve(HVfun1, vl_init, 1,:LM,Arnoldi());
        @assert maximum(abs.(eu)) == abs(eu[1])
        eu=eu[1];
        ev=ev[1];
        return ev,eu
    elseif type=="mpo_A"
        HVfun2(x)=HV_L_tensor(x,A,mpo);
        vl_init = permute(TensorMap(randn, space(A,1)*space(mpo,1),space(mpo,1)*space(A,1)), (1,2,3,4,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        eu,ev=eigsolve(HVfun2, vl_init, 1,:LM,Arnoldi());
        @assert maximum(abs.(eu)) == abs(eu[1])
        eu=eu[1];
        ev=ev[1];
        return ev,eu
    end
end

function right_eigenvector(A,mpo,type)

    if type=="A"
        HVfun1(x)=HV_R_tensor(x,A,[]);
        vr_init = permute(TensorMap(randn, space(A,2), space(A,2)), (1,2,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        eu,ev=eigsolve(HVfun1, vr_init, 1,:LM,Arnoldi());
        @assert maximum(abs.(eu)) == abs(eu[1])
        eu=eu[1];
        ev=ev[1];
        return ev,eu
    elseif type=="mpo_A"
        HVfun2(x)=HV_R_tensor(x,A,mpo);
        vr_init = permute(TensorMap(randn, space(A,2)*space(mpo,3),space(mpo,3)*space(A,2)), (1,2,3,4,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        eu,ev=eigsolve(HVfun2, vr_init, 1,:LM,Arnoldi());
        @assert maximum(abs.(eu)) == abs(eu[1])
        eu=eu[1];
        ev=ev[1];
        return ev,eu
    end
end



function left_eigenvalue(A1,A2,n)
    function HV_L_A1A2(vl,A1,A2)
        if numind(vl)==3
            @tensor vl[:]:=vl[-1,3,1]*A1'[3,-2,2]*A2[1,-3,2];
        elseif numind(vl)==2
            @tensor vl[:]:=vl[3,1]*A1'[3,-2,2]*A2[1,-3,2];
        end
    return vl
    end
    HVA1A2fun(x)=HV_L_A1A2(x,A1,A2);
    vl_init = permute(TensorMap(randn, space(A1,1), space(A2,1)), (1,2,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
    eu,ev=eigsolve(HVA1A2fun, vl_init, 1,:LM,Arnoldi());
    @assert maximum(abs.(eu)) == abs(eu[1])
    eu=eu[1:n];
    return eu
end

left_eigenvalue (generic function with 1 method)

In [170]:
function impo_imps(mpo,A)
    unitary_mpo_A=unitary(fuse(space(mpo,1)⊗space(A,1)), space(mpo,1)⊗space(A,1));
    @tensor mpo_A[:]:=unitary_mpo_A[-1,2,1]*mpo[2,3,4,-3]*A[1,5,3]*unitary_mpo_A'[4,5,-2];
    return mpo_A
end

impo_imps (generic function with 1 method)

In [171]:
function itebd_ite(mpo,A,W,type)
    if type=="A"
        mpo_A=impo_imps(mpo,A);
        vl,_=left_eigenvector(mpo_A,[],"A");
        eu,ev=eig(permute(vl,(1,),(2,)));
        unitary_L=sqrt(eu)*ev';
        unitary_R=ev*pinv(sqrt(eu));
        @tensor AL[:]:=unitary_L[-1,1]*mpo_A[1,2,-3]*unitary_R[2,-2];
    elseif type=="mpo_A"
        vl,_=left_eigenvector(A,mpo,"mpo_A");
        eu,ev=eig(permute(vl,(2,1,),(3,4,)));
        unitary_L=sqrt(eu)*ev';
        unitary_R=ev*pinv(sqrt(eu));
        @tensor AL[:]:=unitary_L[-1,2,1]*mpo[2,3,4,-3]*A[1,5,3]*unitary_R[4,5,-2];
    end

    vr,_=right_eigenvector(AL,[],"A");

    U,S,_=tsvd(permute(vr,(1,),(2,)),trunc=truncdim(W));
    #display(diag(convert(Array,S)))
    @tensor A_trun[:]:=U[1,-1]*AL[1,2,-3]*U'[-2,2];

    return A_trun
end


itebd_ite (generic function with 1 method)

In [172]:
function left_right_normalize(A)
    #left normalize
    vl,_=left_eigenvector(A,[],"A");
    eu,ev=eig(permute(vl,(1,),(2,)));
    unitary_L=sqrt(eu)*ev';
    unitary_R=ev*pinv(sqrt(eu));
    @tensor AL[:]:=unitary_L[-1,1]*A[1,2,-3]*unitary_R[2,-2];
    #transform right fixed point to be diagonal
    vr,eu=right_eigenvector(AL,[],"A");
    U,S,_=tsvd(permute(vr,(1,),(2,)));
    @tensor AL[:]:=U[1,-1]*AL[1,2,-3]*U'[-2,2];
    AL=AL/sqrt(eu);
    return AL
end


left_right_normalize (generic function with 1 method)

In [174]:
function ITEBD_boundary_groundstate(O1,O2,W,A_init,method)
# ITEBD_boundary_groundstate(O1,O2,D,inv_tol):
    #use power method to obtain dominant boundary imps of nonhermitian MPO
    if method=="OO"
        @tensor OO[:]:=O1[-2,-3,-5,1]*O2[-1,1,-4,-6];
        U_fuse_chichi=unitary(fuse(space(OO,1)⊗ space(OO,2)),space(OO,1)⊗ space(OO,2));
        @tensor OO[:]:=U_fuse_chichi[-1,1,2]*OO[1,2,-2,3,4,-4]*U_fuse_chichi'[3,4,-3];
    end
    #Initial state
    if A_init==[]
        mps_virtual=SU₂Space(0=>1,1/2=>1,1=>1);mps_phy=space(O1,2);
        A_init=permute(TensorMap(randn, mps_virtual*mps_virtual', mps_phy),(1,2,3,),());
    else 
    end
    AL=left_right_normalize(A_init);
    
    for cp=1:30
        AL_old=AL;
        @time if method=="O_O"
            A_trun_intermed=itebd_ite(O1,AL,3*W,"mpo_A");
            A_trun=itebd_ite(O2,A_trun_intermed,W,"mpo_A");
            AL=left_right_normalize(A_trun);
        elseif method=="OO"
            A_trun=itebd_ite(OO,AL,W,"mpo_A");#this "mpo_A" option is slightly faster when bond dimension is large
        end
        AL=left_right_normalize(A_trun);

        # vv,ee=right_eigenvector(AL,[],"A");
        # display(ee)
        # u,s,v=tsvd(permute(vv,(1,),(2,)));
        # display(sort(diag(convert(Array,s)),rev=true))
        
        #dominant eigenvalue of transfer matrix
        E=left_eigenvalue(impo_imps(O2,impo_imps(O1,AL)),AL,1)[1]
        ov=abs(left_eigenvalue(AL,AL_old,1)[1]);
        println("E="*string(E)*", "*"ov="*string(ov))
        if abs(ov-1)<1e-8
            break
        end
    
    end
    return AL,A_init
end

ITEBD_boundary_groundstate (generic function with 1 method)

In [175]:
mps_virtual=SU₂Space(0=>1,1/2=>1,1=>1);mps_phy=space(O1,2);
A_init=permute(TensorMap(randn, mps_virtual*mps_virtual', mps_phy),(1,2,3,),());



In [176]:
#Ag,A_init=ITEBD_boundary_groundstate(O1,O2,W,A_init,"OO");


In [177]:

Ag,A_init=ITEBD_boundary_groundstate(O1,O2,W,A_init,"O_O");

  1.204511 seconds (4.12 M allocations: 258.514 MiB, 8.85% gc time, 78.44% compilation time)
E=0.1530650490409543 + 0.004715588332468271im, ov=0.4352098023230009
  0.175798 seconds (923.34 k allocations: 83.464 MiB, 17.86% compilation time)
E=0.15673978933839983 + 0.0017234241084421176im, ov=0.9924292631481638
  0.142840 seconds (830.90 k allocations: 73.451 MiB)
E=0.15739718107944875 + 0.000708861155094484im, ov=0.9989815693667304
  0.146970 seconds (828.27 k allocations: 73.149 MiB)
E=0.15760477918177818 + 0.00038155404704064144im, ov=0.9997594649302831
  0.142131 seconds (828.27 k allocations: 73.149 MiB)
E=0.15769072891364075 + 0.00026925442218274365im, ov=0.999880559732363
  0.280092 seconds (829.83 k allocations: 73.322 MiB, 47.06% gc time)
E=0.1577286483849104 + 0.0002166198810137493im, ov=0.9999145169113665
  0.132558 seconds (828.27 k allocations: 73.149 MiB)
E=0.15774405987788576 + 0.00017665608215103323im, ov=0.9999370236300039
  0.141200 seconds (831.39 k allocations: 73.49

In [178]:
#save initial CTM to compare with other codes
matwrite("itebd_matfile.mat", Dict(
    "O1" => convert(Array,O1),
    "O2" => convert(Array,O2),
    "A_init" => convert(Array,A_init),
    "Ag" => convert(Array,Ag)
); compress = false)



In [179]:
#save data
O1_dict=convert(Dict,O1);
O2_dict=convert(Dict,O2);
Ag_dict=convert(Dict,Ag);
save("itebd_data.jld", "O1_dict", O1_dict,"O2_dict", O2_dict,"Ag_dict", Ag_dict);




In [180]:
#load data
Ag_dict=load("itebd_data.jld")["Ag_dict"];
O1_dict=load("itebd_data.jld")["O1_dict"];
O2_dict=load("itebd_data.jld")["O2_dict"];
Ag=convert(TensorMap,Ag_dict);
O1=convert(TensorMap,O1_dict);
O2=convert(TensorMap,O2_dict);


In [181]:

space_AOA=fuse(space(Ag,1)'⊗space(O2,1)'⊗space(O1,1)⊗ space(Ag,1));
display(space_AOA)
display(dim(space_AOA))
space_AA=fuse(space(Ag,1)'⊗ space(Ag,1));
display(space_AA)
display(dim(space_AA))

Rep[SU₂](0=>129, 1/2=>196, 1=>185, 3/2=>124, 2=>61, 5/2=>20, 3=>4)

2025

Rep[SU₂](0=>9, 1/2=>12, 1=>9, 3/2=>4, 2=>1)

81

In [182]:
# W=20;chi=20;
# (Rep[SU₂](0=>3, 1/2=>3, 1=>2, 3/2=>1)' ⊗ Rep[SU₂](0=>3, 1/2=>3, 1=>2, 3/2=>1) ⊗ Rep[SU₂](0=>1, 1/2=>2, 1=>1)') ← ProductSpace{GradedSpace{SU2Irrep, TensorKit.SortedVectorDict{SU2Irrep, Int64}}, 0}()
# Rep[SU₂](0=>4999, 1/2=>8314, 1=>9146, 3/2=>7850, 2=>5503, 5/2=>3190, 3=>1521, 7/2=>584, 4=>173, 9/2=>36, 5=>4)

In [183]:
for s in sectors(space_AOA)
  display(s)
  @show s, dim(space_AOA, s), dim(s)
end


Irrep[SU₂](0)

Irrep[SU₂](1/2)

Irrep[SU₂](1)

Irrep[SU₂](3/2)

Irrep[SU₂](2)

Irrep[SU₂](5/2)

Irrep[SU₂](3)

(s, dim(space_AOA, s), dim(s)) = (Irrep[SU₂](0), 129, 1)
(s, dim(space_AOA, s), dim(s)) = (Irrep[SU₂](1/2), 196, 2)
(s, dim(space_AOA, s), dim(s)) = (Irrep[SU₂](1), 185, 3)
(s, dim(space_AOA, s), dim(s)) = (Irrep[SU₂](3/2), 124, 4)
(s, dim(space_AOA, s), dim(s)) = (Irrep[SU₂](2), 61, 5)
(s, dim(space_AOA, s), dim(s)) = (Irrep[SU₂](5/2), 20, 6)
(s, dim(space_AOA, s), dim(s)) = (Irrep[SU₂](3), 4, 7)


In [184]:
AOA_sec=collect(sectors(space_AOA))
AOA_sec[1]
AA_sec=collect(sectors(space_AA))
AA_sec[1]


Irrep[SU₂](0)

In [185]:
@tensor OO[:]:=O1[-2,-3,-5,1]*O2[-1,1,-4,-6];
U_fuse_chichi=unitary(fuse(space(OO,1)⊗ space(OO,2)),space(OO,1)⊗ space(OO,2));
@tensor OO[:]:=U_fuse_chichi[-1,1,2]*OO[1,2,-2,3,4,-4]*U_fuse_chichi'[3,4,-3];
display(space(OO))


(Rep[SU₂](0=>5, 1/2=>4, 1=>4) ⊗ Rep[SU₂](0=>1, 1/2=>1, 1=>1) ⊗ Rep[SU₂](0=>5, 1/2=>4, 1=>4)' ⊗ Rep[SU₂](0=>1, 1/2=>1, 1=>1)') ← ProductSpace{GradedSpace{SU2Irrep, TensorKit.SortedVectorDict{SU2Irrep, Int64}}, 0}()

In [186]:
# #attention: when the dimension is slightly large, converting dense matrix to tensormap is extremely difficult. It's better to define tensormap from Dict
# using TensorKit
# sp=Rep[SU₂](0=>129, 1/2=>196, 1=>185, 3/2=>124, 2=>61, 5/2=>20, 3=>4); #out of memory error for this dimension
# sp=Rep[SU₂](0=>12, 1/2=>19, 1=>15, 3/2=>14, 2=>61, 5/2=>20, 3=>4); # it takes long time
# tt=TensorMap(randn,sp←sp);
# tt_dense=convert(Array,tt);
# tt_new=TensorMap(tt_dense,sp,sp);
function combine_singlespin_sector_bruteforce(E_set,vL_set,vR_set,spin_set)
    E_set_new=copy(euL_set);
    vL_set_new=copy(evL_set);
    vR_set_new=copy(evR_set);
    spin_set_new=copy(SPIN_eig_set);
    for cc=1:length(E_set)
        E=E_set[cc];
        vL=vL_set[cc];
        vR=vR_set[cc];
        spin=spin_set[cc];
        if length(vL)>0
            single_size=size(convert(Array,vL[1]));
            if length(single_size)==4
                vL_grouped=zeros(Complex{Float64},single_size[1]*length(vL),single_size[2]*single_size[3]*single_size[4]);
                vR_grouped=zeros(Complex{Float64},single_size[2]*single_size[3]*single_size[4],single_size[1]*length(vL));
                E_grouped=zeros(Complex{Float64},single_size[1]*length(vL),single_size[1]*length(vL));
                spin_dim=single_size[1];
                Unitary=unitary(domain(vL[1]),fuse(domain(vL[1])));
                for cs=1:length(E)
                    vL_grouped[spin_dim*(cs-1)+1:spin_dim*cs,:]=convert(Array,vL[cs]*Unitary);
                    vR_grouped[:,spin_dim*(cs-1)+1:spin_dim*cs]=convert(Array,Unitary'*vR[cs]);
                    E_grouped[spin_dim*(cs-1)+1:spin_dim*cs,spin_dim*(cs-1)+1:spin_dim*cs]=E[cs]*Matrix(I, spin_dim,spin_dim);
                end
                space_single_spin=SU₂Space(spin[1]=>length(E));

                vL_grouped=TensorMap(vL_grouped,space_single_spin←fuse(domain(vL[1])));
                vR_grouped=TensorMap(vR_grouped,fuse(domain(vL[1])) ← space_single_spin);
                E_grouped=TensorMap(E_grouped,space_single_spin ← space_single_spin);

                vL_grouped=vL_grouped*Unitary';
                vR_grouped=Unitary * vR_grouped;

                vL_set_new[cc]=vL_grouped;
                vR_set_new[cc]=vR_grouped;
                E_set_new[cc]=E_grouped;
                spin_set_new[cc]=spin_set[cc][1]
                # display(Base.summarysize(vL_grouped))
                # display(space(vL_grouped))
                # display(Base.summarysize(vL))
            elseif length(single_size)==3
            end
        end
    end
    return E_set_new,vL_set_new,vR_set_new,spin_set_new
end


combine_singlespin_sector_bruteforce (generic function with 1 method)

In [188]:
function combine_singlespin_sector(E_set,vL_set,vR_set,spin_set,is_eig)
    E_set_new=copy(E_set);
    vL_set_new=copy(vL_set);
    vR_set_new=copy(vR_set);
    spin_set_new=copy(spin_set);
    for cc=1:length(E_set)
        E=E_set[cc];
        vL=vL_set[cc];
        vR=vR_set[cc];
        spin=spin_set[cc];
        if length(vL)>0

            sec=Irrep[SU₂](spin[1]);
            space_single_spin=SU₂Space(spin[1]=>length(E));
            dim_full=dim(fuse(domain(vL[1])),sec);
            vL_grouped=zeros(Complex{Float64},length(vL),dim_full);
            vR_grouped=zeros(Complex{Float64},dim_full,length(vL));
            E_grouped=zeros(Complex{Float64},length(vL),length(vL));

            Unitary=unitary(domain(vL[1]),fuse(domain(vL[1])));
            for cs=1:length(E)
                vL_grouped[cs,:]=convert(Dict,vL[cs]*Unitary)[:data][string(sec)];
                vR_grouped[:,cs]=convert(Dict,Unitary'*vR[cs])[:data][string(sec)];
                E_grouped[cs,cs]=E[cs];
            end

            if is_eig
                Q=zeros(Complex{Float64},length(E),length(E))
                for ca=1:length(E)
                    for cb=1:length(E)
                        Q[ca,cb]=tr(vL[ca]*vR[cb])
                    end
                end
                #Q=vL_grouped*vR_grouped;#the normalization of this line is inccorect, due to the dimension of spin space
                vL_grouped=pinv(Q)*vL_grouped;
            end

            vL_grouped_rand=TensorMap(randn,space_single_spin←fuse(domain(vL[1])));
            vR_grouped_rand=TensorMap(randn,fuse(domain(vL[1])) ← space_single_spin);
            E_grouped_rand=TensorMap(randn,space_single_spin ← space_single_spin);

            vL_grouped_dict=convert(Dict,vL_grouped_rand);
            vL_grouped_dict[:data][string(sec)]=vL_grouped;
            vL_grouped=convert(TensorMap,vL_grouped_dict);
            vL_grouped=vL_grouped*Unitary';

            vR_grouped_dict=convert(Dict,vR_grouped_rand);
            vR_grouped_dict[:data][string(sec)]=vR_grouped;
            vR_grouped=convert(TensorMap,vR_grouped_dict);
            vR_grouped=Unitary*vR_grouped;

            E_grouped_dict=convert(Dict,E_grouped_rand);
            E_grouped_dict[:data][string(sec)]=E_grouped;
            E_grouped=convert(TensorMap,E_grouped_dict);

            vL_set_new[cc]=vL_grouped;
            vR_set_new[cc]=vR_grouped;
            E_set_new[cc]=E_grouped;
            spin_set_new[cc]=spin_set_new[cc][1]
            # display(Base.summarysize(vL_grouped))
            # display(space(vL_grouped))
            # display(Base.summarysize(vL))

        end
    end
    return E_set_new,vL_set_new,vR_set_new,spin_set_new
end




combine_singlespin_sector (generic function with 2 methods)

In [189]:

function truncate_sectors(n,eu_set,ev_set,ev_set2,spin_set)
    eu_full=eu_set[1]
    dim_full=2*spin_set[1].+1
    for cc=2:length(eu_set)
        eu_full=vcat(eu_full, eu_set[cc])
        dim_full=vcat(dim_full, 2*spin_set[cc].+1)
    end
    dim_full=Int.(round.(dim_full))

    order=sortperm(abs.(eu_full),rev=true);
    eu_full_sorted=eu_full[order];
    dim_full_sorted=dim_full[order];

    total_size=dim_full_sorted[1];
    old_value=eu_full_sorted[1];
    for cc=2:length(eu_full_sorted)
        total_size=total_size+dim_full_sorted[cc];
        new_value=eu_full_sorted[cc];
        if total_size>n
            break;
        end
        old_value=new_value;
    end

    for cc=1:length(eu_set)
        spins=spin_set[cc];
        eu=eu_set[cc];
        ev=ev_set[cc];
        inds=findall(x->abs(x)>=abs(old_value), abs.(eu));
        eu=eu[inds];
        ev=ev[inds];
        spins=spins[inds];
        eu_set[cc]=eu;
        ev_set[cc]=ev;
        spin_set[cc]=spins;
        if ~(ev_set2==[])
            ev2=ev_set2[cc];
            ev2=ev2[inds];
            ev_set2[cc]=ev2;
        end
    end
return eu_set,ev_set,ev_set2,spin_set

end


truncate_sectors (generic function with 1 method)

In [190]:
function FLR_eig(A,O,n,full_space,sector_set)
    SPIN_set=Vector{Any}(undef, length(sector_set));
    euL_set=Vector{Any}(undef, length(sector_set));
    evL_set=Vector{Any}(undef, length(sector_set));
    euR_set=Vector{Any}(undef, length(sector_set));
    evR_set=Vector{Any}(undef, length(sector_set));
    println("FLR_eig: ")
    for cc=1:length(sector_set)
        sec=sector_set[cc];
        spin=(dim(sec)-1)/2;
        n_eff=Int(round(n/dim(sec)))+1
        println("spin "*string(spin))

        FLR_eig_L(x)=HV_FL(x,A,O);
        vl_init=permute(TensorMap(randn, SU₂Space(spin=>1),space(A',1)*space(O,1)*space(A,1)), (1,2,3,4,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        try
            euL,evL,info=eigsolve(FLR_eig_L, vl_init, minimum([n_eff,dim(full_space,sec)]),:LM, Arnoldi(krylovdim=minimum([n_eff,dim(full_space,sec)])*2));
            @assert info.converged >= minimum([n_eff,dim(full_space,sec)])
        catch e
            println("Number of eigenvalues obtained are not enough, use smaller tol")
            euL,evL,info=eigsolve(FLR_eig_L, vl_init, minimum([n_eff,dim(full_space,sec)]),:LM, Arnoldi(krylovdim=minimum([n_eff,dim(full_space,sec)])*2, tol=1e-14));
            @assert info.converged >= minimum([n_eff,dim(full_space,sec)])
        end
        @assert abs.(euL)==sort(abs.(euL), rev=true)
        euL=euL[1:minimum([length(euL),n_eff,dim(full_space,sec)])];
        evL=evL[1:minimum([length(evL),n_eff,dim(full_space,sec)])];
        euL_set[cc]=euL;
        evLL=evL;
        evL=Vector{Any}(undef, length(evLL));
        for cs=1:length(evL)
            evL[cs]=permute(evLL[cs],(1,),(2,3,4,))
        end
        evL_set[cc]=evL;
        SPIN_set[cc]=ones(length(euL))*(dim(sec)-1)/2;
        
        FLR_eig_R(x)=HV_FR(x,A,O);
        vr_init=permute(TensorMap(randn, space(A',2)'*space(O,3)'*space(A,2)',SU₂Space(spin=>1)), (1,2,3,4,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        try
            euR,evR,info=eigsolve(FLR_eig_R, vr_init, minimum([n_eff,dim(full_space,sec)]),:LM, Arnoldi(krylovdim=minimum([n_eff,dim(full_space,sec)])*2));
            @assert info.converged >= minimum([n_eff,dim(full_space,sec)])
        catch e
            println("Number of eigenvalues obtained are not enough, use smaller tol")
            euR,evR,info=eigsolve(FLR_eig_R, vr_init, minimum([n_eff,dim(full_space,sec)]),:LM, Arnoldi(krylovdim=minimum([n_eff,dim(full_space,sec)])*2, tol=1e-14));
            @assert info.converged >= minimum([n_eff,dim(full_space,sec)])
        end
        @assert abs.(euR)==sort(abs.(euR), rev=true)
        euR=euR[1:minimum([length(euR),n_eff,dim(full_space,sec)])];
        evR=evR[1:minimum([length(evR),n_eff,dim(full_space,sec)])];
        euR_set[cc]=euR;
        evRR=evR;
        evR=Vector{Any}(undef, length(evRR));
        for cs=1:length(evR)
            evR[cs]=permute(evRR[cs],(1,2,3,),(4,))
        end
        evR_set[cc]=evR;

        @assert (norm(abs.(euL)-abs.(euR))/norm(euL))<1e-8
    end
    return euL_set,evL_set,euR_set,evR_set,SPIN_set     
end
        
function HV_FL(vl,A,mpo)
    @tensor vl[:]:=vl[-1,1,3,5]*A'[1,-2,2]*mpo[3,4,-3,2]*A[5,-4,4];
    return vl
end

function HV_FR(vr,A,mpo)
    @tensor vr[:]:=A'[-1,1,2]*mpo[-2,4,3,2]*A[-3,5,4]*vr[1,3,5,-4];
    return vr
end
 
        


HV_FR (generic function with 1 method)

In [192]:
function GLR_eig(A,n,full_space,sector_set)
    spin_set=Vector{Any}(undef, length(sector_set));
    eul_set=Vector{Any}(undef, length(sector_set));
    evl_set=Vector{Any}(undef, length(sector_set));
    eur_set=Vector{Any}(undef, length(sector_set));
    evr_set=Vector{Any}(undef, length(sector_set));
    println("GLR_eig: ")
    for cc=1:length(sector_set)
        sec=sector_set[cc];
        spin=(dim(sec)-1)/2;
        n_eff=Int(round(n/dim(sec)))+1
        println("spin "*string(spin))
        
        GLR_eig_L(x)=HV_L_tensor(x,A,[]);
        vl_init=permute(TensorMap(randn, SU₂Space(spin=>1),space(A',1)*space(A,1)), (1,2,3,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        try
            eul,evl,info=eigsolve(GLR_eig_L, vl_init, minimum([n_eff,dim(full_space,sec)]),:LM,Arnoldi(krylovdim=minimum([n_eff,dim(full_space,sec)])*2));
            @assert info.converged >= minimum([n_eff,dim(full_space,sec)])
        catch e
            println("Number of eigenvalues obtained are not enough, use smaller tol")
            eul,evl,info=eigsolve(GLR_eig_L, vl_init, minimum([n_eff,dim(full_space,sec)]),:LM,Arnoldi(krylovdim=minimum([n_eff,dim(full_space,sec)])*2, tol=1e-14));
            @assert info.converged >= minimum([n_eff,dim(full_space,sec)])
        end
        @assert abs.(eul)==sort(abs.(eul), rev=true)
        
        eul=eul[1:minimum([length(eul),n_eff,dim(full_space,sec)])];
        evl=evl[1:minimum([length(evl),n_eff,dim(full_space,sec)])];
        eul_set[cc]=eul;
        evll=evl;
        evl=Vector{Any}(undef, length(evll));
        for cs=1:length(evl)
            evl[cs]=permute(evll[cs],(1,),(2,3,))
        end
        evl_set[cc]=evl;
        spin_set[cc]=ones(length(eul))*(dim(sec)-1)/2;

        GLR_eig_R(x)=HV_R_tensor(x,A,[]);
        vr_init=permute(TensorMap(randn, space(A',2)'*space(A,2)',SU₂Space(spin=>1)), (1,2,3,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        try
            eur,evr,info=eigsolve(GLR_eig_R, vr_init, minimum([n_eff,dim(full_space,sec)]),:LM,Arnoldi(krylovdim=minimum([n_eff,dim(full_space,sec)])*2));
            @assert info.converged >= minimum([n_eff,dim(full_space,sec)])
        catch e
            println("Number of eigenvalues obtained are not enough, use smaller tol")
            eur,evr,info=eigsolve(GLR_eig_R, vr_init, minimum([n_eff,dim(full_space,sec)]),:LM,Arnoldi(krylovdim=minimum([n_eff,dim(full_space,sec)])*2, tol=1e-14));
            @assert info.converged >= minimum([n_eff,dim(full_space,sec)])
        end
        @assert abs.(eur)==sort(abs.(eur), rev=true)
        eur=eur[1:minimum([length(eur),n_eff,dim(full_space,sec)])];
        evr=evr[1:minimum([length(evl),n_eff,dim(full_space,sec)])];
        eur_set[cc]=eur;
        evrr=evr;
        evr=Vector{Any}(undef, length(evrr));
        for cs=1:length(evr)
            evr[cs]=permute(evrr[cs],(1,2,),(3,))
        end
        evr_set[cc]=evr;

        @assert (norm(abs.(eul)-abs.(eur))/norm(eul))<1e-8

    end
    return eul_set,evl_set,eur_set,evr_set,spin_set   
end
    
    
    

GLR_eig (generic function with 1 method)

In [193]:
function FLR_svd(A,O,pow,n,full_space,sector_set)
    SPIN_set=Vector{Any}(undef, length(sector_set));
    S_set=Vector{Any}(undef, length(sector_set));
    U_set=Vector{Any}(undef, length(sector_set));
    Vh_set=Vector{Any}(undef, length(sector_set));
    println("FLR_svd: ")
    for cc=1:length(sector_set)
        sec=sector_set[cc];
        spin=(dim(sec)-1)/2;
        n_eff=Int(round(n/dim(sec)))+1
        println("spin "*string(spin))

        FLR_svd_R(x)=HV_FR_pow(x,A,O,pow);
        FLR_svd_R_conj(x)=HV_FR_conj_pow(x,A,O,pow);
        vl_init=permute(TensorMap(randn, SU₂Space(spin=>1),space(A',1)*space(O,1)*space(A,1)), (1,2,3,4,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        vr_init=permute(TensorMap(randn, space(A',2)'*space(O,3)'*space(A,2)',SU₂Space(spin=>1)), (1,2,3,4,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        try
            S,U,V,info=svdsolve((FLR_svd_R,FLR_svd_R_conj), vr_init, minimum([n_eff,dim(full_space,sec)]),:LR, krylovdim=minimum([n_eff,dim(full_space,sec)])*2);
            @assert info.converged >= minimum([n_eff,dim(full_space,sec)])
        catch e
            println("Number of singular values obtained are not enough, use smaller tol")
            S,U,V,info=svdsolve((FLR_svd_R,FLR_svd_R_conj), vr_init, minimum([n_eff,dim(full_space,sec)]),:LR, krylovdim=minimum([n_eff,dim(full_space,sec)])*2, tol=1e-14);
            @assert info.converged >= minimum([n_eff,dim(full_space,sec)])
        end
        @assert abs.(S)==sort(abs.(S), rev=true)
        S=S[1:minimum([length(S),n_eff,dim(full_space,sec)])];
        U=U[1:minimum([length(U),n_eff,dim(full_space,sec)])];
        V=V[1:minimum([length(V),n_eff,dim(full_space,sec)])];

        UU=U;
        U=Vector{Any}(undef, length(UU));
        for cs=1:length(U)
            U[cs]=permute(UU[cs],(1,2,3,),(4,))
        end

        Vh=Vector{Any}(undef, length(V));
        for cs=1:length(Vh)
            vvr=V[cs];
            Vh[cs]=permute(vvr',(4,),(1,2,3,));
        end
        S_set[cc]=S;
        U_set[cc]=U;
        Vh_set[cc]=Vh;
        SPIN_set[cc]=ones(length(S))*(dim(sec)-1)/2;
        
    end
    return S_set,U_set,Vh_set,SPIN_set     
end
            

function HV_FR_pow(vr,A,mpo,pow)
    for cc=1:pow
        vr=HV_FR(vr,A,mpo);
    end
    return vr
end
function HV_FR_conj(vr,A,mpo)
    @tensor vr[:]:=A[1,-1,2]*mpo'[3,4,-2,2]*A'[5,-3,4]*vr[1,3,5,-4];
    return vr
end
function HV_FR_conj_pow(vr,A,mpo,pow)
    for cc=1:pow
        vr=HV_FR_conj(vr,A,mpo);
    end
    return vr
end
    

HV_FR_conj_pow (generic function with 1 method)

In [194]:
function GLR_svd(A,pow,n,full_space,sector_set)
    spin_set=Vector{Any}(undef, length(sector_set));
    s_set=Vector{Any}(undef, length(sector_set));
    u_set=Vector{Any}(undef, length(sector_set));
    vh_set=Vector{Any}(undef, length(sector_set));
    A_noise=TensorMap(randn, codomain(A),domain(A));
    A_noise=A_noise/norm(A_noise)*norm(A)*1e-9;
    println("GLR_svd: ")
    for cc=1:length(sector_set)
        sec=sector_set[cc];
        spin=(dim(sec)-1)/2;
        n_eff=Int(round(n/dim(sec)))+1
        println("spin "*string(spin))


        vr_init=permute(TensorMap(randn, space(A',2)'*space(A,2)',SU₂Space(spin=>1)), (1,2,3,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
        try
            GLR_svd_R(x)=HV_R_pow(x,A,pow,A_noise*0);
            GLR_svd_R_conj(x)=HV_R_conj_pow(x,A,pow,A_noise*0);
            s,u,v,info=svdsolve((GLR_svd_R,GLR_svd_R_conj), vr_init, minimum([n_eff,dim(full_space,sec)]),:LR, krylovdim=minimum([n_eff,dim(full_space,sec)])*2);
            @assert info.converged >= minimum([n_eff,dim(full_space,sec)])    
        catch e
            GLR_svd_R(x)=HV_R_pow(x,A,pow,A_noise);
            GLR_svd_R_conj(x)=HV_R_conj_pow(x,A,pow,A_noise);
            println("Number of singular values obtained are not enough, add noise, increase Krylov subspace and use smaller tol")
            s,u,v,info=svdsolve((GLR_svd_R,GLR_svd_R_conj), vr_init, minimum([n_eff,dim(full_space,sec)]),:LR, krylovdim=minimum([n_eff,dim(full_space,sec)])*4, tol=1e-14);
            @assert info.converged >= minimum([n_eff,dim(full_space,sec)])
        end
        @assert abs.(s)==sort(abs.(s), rev=true)
    
        s=s[1:minimum([length(s),n_eff,dim(full_space,sec)])];
        u=u[1:minimum([length(u),n_eff,dim(full_space,sec)])];
        v=v[1:minimum([length(v),n_eff,dim(full_space,sec)])];

        uu=u;
        u=Vector{Any}(undef, length(u));
        for cs=1:length(u)
            u[cs]=permute(uu[cs],(1,2,),(3,))
        end

        vh=Vector{Any}(undef, length(v));
        for cs=1:length(vh)
            vvr=v[cs];
            vh[cs]=permute(vvr',(3,),(1,2,));
        end
        s_set[cc]=s;
        u_set[cc]=u;
        vh_set[cc]=vh;
        spin_set[cc]=ones(length(s))*(dim(sec)-1)/2;
        
    end
    return s_set,u_set,vh_set,spin_set     
end

function HV_R_pow(vr,A,pow,A_noise)
    A_=A+A_noise;
    for cc=1:pow
        @tensor vr[:]:=A_'[-1,1,2]*A[-2,3,2]*vr[1,3,-3];
    end
    return vr
end

function HV_R_conj_pow(vr,A,pow,A_noise)
    A_=A+A_noise;
    for cc=1:pow
        @tensor vr[:]:=A_[1,-1,2]*A'[3,-2,2]*vr[1,3,-3];
    end
    return vr
end



HV_R_conj_pow (generic function with 1 method)

In [195]:

function TransfOp_decom(A,O,pow,Dtrun_init,Dtrun_max,trun_tol,method)
    Dstep=50;
    Dtrun=Dtrun_init;
    if method=="eigenvalue_FLR"
        for cs=1:round((Dtrun_max-Dtrun_init)/Dstep)+1
            evL,lambdaL=F_L(A,O,Dtrun);
            evR,lambdaR=F_R(A,O,Dtrun);
    
            eu=abs(lambdaL);
            eu=eu/max(eu); 
            eu=eu.^pow;
            disp(['method=',method,', Dtrun=',num2str(Dtrun),', minimal eigenvalue: ',num2str(min(eu))])
            if min(eu)<trun_tol
                break;
            else
                if Dtrun>(size(A,1)^2)*(size(O,1)^2)
                    disp('Dtrun exceeds matrix size')
                    break;
                end
                Dtrun=Dtrun+Dstep;
            end
        end
        [eu_sorted,order]=sort(eu,'descend');
        evL=evL(:,order);
        lambdaL=lambdaL(order);
        evR=evR(:,order);
        lambdaR=lambdaR(order);
        
        posit=find(eu_sorted>trun_tol);
        evL=evL(:,posit);
        lambdaL=lambdaL(posit);
        evR=evR(:,posit);
        lambdaR=lambdaR(posit);
        
        return evL,lambdaL,evR,lambdaR

    elseif method=="eigenvalue_GLR"
        for cs=1:round((Dtrun_max-Dtrun_init)/Dstep)+1
            evl,lambdal=G_L(A,A,Dtrun);
            evr,lambdar=G_R(A,A,Dtrun);
    
            eu=abs(lambdal);
            eu=eu/max(eu); 
            eu=eu.^pow;
            disp(['method=',method,', Dtrun=',num2str(Dtrun),', minimal eigenvalue: ',num2str(min(eu))])
            if min(eu)<trun_tol
                break;
            else
                if Dtrun>size(A,1)^2
                    disp('Dtrun exceeds matrix size')
                    break;
                end
                Dtrun=Dtrun+Dstep;
            end
        end
        [eu_sorted,order]=sort(eu,'descend');
        evl=evl(:,order);
        lambdal=lambdal(order);
        evr=evr(:,order);
        lambdar=lambdar(order);
        
        posit=find(eu_sorted>trun_tol);
        evl=evl(:,posit);
        lambdal=lambdal(posit);
        evr=evr(:,posit);
        lambdar=lambdar(posit);
        
        return evl,lambdal,evr,lambdar
    elseif method=="svd_GLR"
        for cs=1:round((Dtrun_max-Dtrun_init)/Dstep)+1
            [u,s,v]=GLR_svd(A,pow,Dtrun);
            vt=v';
            eu=abs(diag(s));
            eu=eu/max(eu); 
            disp(['method=',method,', Dtrun=',num2str(Dtrun),', minimal singularvalue: ',num2str(min(eu))])
            if min(eu)<trun_tol
                break;
            else
                if Dtrun>(size(A,1)^2)
                    disp('Dtrun exceeds matrix size')
                    break;
                end
                Dtrun=Dtrun+Dstep;
            end
        end
        [eu_sorted,order]=sort(eu,'descend');
        u=u(:,order);
        s=s(order,order);
        vt=vt(order,:);
    
        posit=find(eu_sorted>trun_tol);
        u=u(:,posit);
        s=s(posit,posit);
        vt=vt(posit,:);
        
        s=diag(s);
        return u,s,vt
    elseif method=="svd_FLR"
        for cs=1:round((Dtrun_max-Dtrun_init)/Dstep)+1
            [U,S,V]=FLR_svd(A,O,pow,Dtrun);
            Vt=V';   
            eu=abs(diag(S));
            eu=eu/max(eu); 
            disp(['method=',method,', Dtrun=',num2str(Dtrun),', minimal singularvalue: ',num2str(min(eu))])
            if min(eu)<trun_tol
                break;
            else
                if Dtrun>(size(A,1)^2)*(size(O,1)^2)
                    disp('Dtrun exceeds matrix size')
                    break;
                end
                Dtrun=Dtrun+Dstep;
            end
        end
        [eu_sorted,order]=sort(eu,'descend');
        U=U(:,order);
        S=S(order,order);
        Vt=Vt(order,:);
        
        posit=find(eu_sorted>trun_tol);
        U=U(:,posit);
        S=S(posit,posit);
        Vt=Vt(posit,:);
        
        
        S=diag(S);
        
        return U,S,Vt
    end
end
    

    
    


ErrorException: syntax: character literal contains multiple characters

In [196]:

euL_set,evL_set,euR_set,evR_set,SPIN_eig_set=FLR_eig(Ag,OO,Dtrun,space_AOA,AOA_sec);
display(size(euL_set))
eul_set,evl_set,eur_set,evr_set,spin_eig_set=GLR_eig(Ag,Dtrun,space_AA,AA_sec);
display(size(eul_set))
S_set,U_set,Vh_set,SPIN_svd_set=FLR_svd(Ag,OO,pow,Dtrun,space_AOA,AOA_sec);
display(size(S_set))
s_set,u_set,vh_set,spin_svd_set=GLR_svd(Ag,pow,Dtrun,space_AA,AA_sec);
display(size(s_set))


euL_set,evL_set,evR_set,SPIN_eig_set=truncate_sectors(Dtrun,euL_set,evL_set,evR_set,SPIN_eig_set);
display(euL_set)
eul_set,evl_set,evr_set,spin_eig_set=truncate_sectors(Dtrun,eul_set,evl_set,evr_set,spin_eig_set);
display(eul_set)
S_set,U_set,Vh_set,SPIN_svd_set=truncate_sectors(Dtrun,S_set,U_set,Vh_set,SPIN_svd_set);
display(S_set)
s_set,u_set,vh_set,spin_svd_set=truncate_sectors(Dtrun,s_set,u_set,vh_set,spin_svd_set);
display(s_set)

(7,)

(5,)

(7,)

(5,)

7-element Vector{Any}:
 ComplexF64[0.1577577388961442 - 1.8843654374603057e-6im, 0.0535664687199532 - 0.08724814121487774im, 0.053562574287501656 + 0.08724842357783105im, 0.09173162414628698 - 7.399530496214209e-6im, 0.06264627206107223 + 1.0343247448011331e-5im, 0.032024325672146775 + 0.053605100727527125im, 0.032016869699938645 - 0.053587468275319676im, 0.056173363498948255 + 1.0880505537477998e-6im, 0.010932516726949589 - 0.04210979658536626im, 0.010928143017160146 + 0.04209629202921178im  …  0.0003920014070806474 - 0.0018115286083894753im, 0.00018052781603104652 + 0.0018263889204132402im, 0.00017693908504361975 - 0.0018261108578554617im, 0.0017807522558686234 - 0.00022574997521938216im, 0.00177662105842727 + 0.00022431472790487706im, 0.0015376826708557432 + 1.3616820218457937e-5im, 0.00039855839603203163 - 0.001176380856126887im, 0.00039709719140317256 + 0.0011732105297074635im, 0.000989101847560368 - 9.949257965037652e-7im, 0.00033894023968505276 + 1.3947493102517843e-6im]
 Comple

5-element Vector{Any}:
 ComplexF64[0.9999999999999973 + 1.1102230246251565e-16im, 0.3379962808499724 - 0.497611299649818im, 0.33799628084997224 + 0.4976112996498179im, 0.5330857954257686 - 1.509426542297036e-17im, 0.28108383407147086 - 1.0998036619746701e-17im, 0.21516723753989928 - 2.061736894043722e-17im, 0.17012205080898676 - 1.5969608439870672e-18im, 0.1681988851459231 + 7.369233431548104e-17im, 0.0364031507094389 - 1.649969241822698e-17im]
 ComplexF64[0.7317570730435999 + 0.36850345956332353im, 0.731757073043599 - 0.3685034595633234im, 0.45019336220961054 - 0.2841385768120297im, 0.4501933622096101 + 0.28413857681202936im, 0.27227318937820094 + 0.15193735019476717im, 0.2722731893782008 - 0.15193735019476715im, 0.2387226682894182 - 0.14168091687930714im, 0.238722668289418 + 0.14168091687930726im, 0.14912504424628642 + 0.11262801983582697im, 0.14912504424628634 - 0.11262801983582708im, 0.12490204321322268 + 0.05097450581684573im, 0.12490204321322247 - 0.05097450581684565im]
 ComplexF

7-element Vector{Any}:
 [0.044684212695000794, 0.033784468263917525, 0.018352909028729183, 0.017264526437359165, 0.012847291278774124, 0.009518535121174152, 0.006918592030123912, 0.004466150065214447, 0.0041190315373507, 0.0035357415378703856  …  2.402348088988193e-6, 2.266291389477186e-6, 2.1264203219253712e-6, 1.962933456830585e-6, 1.5710312367605628e-6, 1.3290031299073674e-6, 1.0555713883578876e-6, 7.66570463419503e-7, 4.9936018750805e-7, 7.70307867195025e-8]
 [0.039603550019052434, 0.037828596140619904, 0.019137415742411026, 0.016529945244107003, 0.014122508475273096, 0.01364822411128442, 0.010300717050496329, 0.00882552696030321, 0.005724106283320943, 0.005135775063152593  …  2.3338535728206376e-6, 2.056561877683264e-6, 1.8332986900336067e-6, 1.7829060490700796e-6, 1.593961461664053e-6, 1.4626938543986664e-6, 1.2106719590196731e-6, 7.921424063286178e-7, 6.158792059762683e-7, 4.4279390499238225e-7]
 [0.03424081518935906, 0.01794913438443319, 0.015626722810191784, 0.0135957721554321

5-element Vector{Any}:
 [1.846734729500663, 0.9532069195679144, 0.33369318743920096, 0.2847883922085106, 0.05039510224212826, 0.03731772222011834, 0.028857269443589212, 0.016440595276183773, 0.0009894722466946202]
 [1.2619524918325162, 1.2619524918325158, 0.2962930193568062, 0.296293019356806, 0.1517565560671654, 0.15175655606716537, 0.041148853716427645, 0.04114885371642759, 0.0357469244484414, 0.03574692444844138, 0.010852873424266775, 0.010852873424266753]
 [1.0943068124535755, 0.29765788137271115, 0.28475423284039614, 0.14794710323477717, 0.1454365022601171, 0.08390769273824625, 0.02487637254348843, 0.0069423374633009, 0.005013188526528343]
 [0.15150186065828344, 0.15150186008192987, 0.0484474255510829, 0.04844742542810276]
 [0.027839923099422667]

FLR_eig: 
spin 0.0
spin 0.5
spin 1.0
spin 1.5
spin 2.0
spin 2.5
spin 3.0
GLR_eig: 
spin 0.0
spin 0.5
spin 1.0
spin 1.5
spin 2.0
FLR_svd: 
spin 0.0
spin 0.5
spin 1.0
spin 1.5
spin 2.0
spin 2.5
spin 3.0
GLR_svd: 
spin 0.0
spin 0.5
spin 1.0
spin 1.5
Number of singular values obtained are not enough, add noise, increase Krylov subspace and use smaller tol
spin 2.0


┌ Warning: Invariant subspace of dimension 2 (up to requested tolerance `tol = 1.0e-12`), which is smaller than the number of requested singular values (i.e. `howmany == 4`); setting `howmany = 2`.
└ @ KrylovKit C:\Users\Lenovo\.julia\packages\KrylovKit\YPiz7\src\eigsolve\svdsolve.jl:142


In [197]:

#check FLR_svd
@tensor M[:]:=Ag'[-1,-4,1]*OO[-2,2,-5,1]*Ag[-3,-6,2];
M=permute(M,(1,2,3,),(4,5,6,));
for cc=1:pow-1
    M=M*M;
end
M_temp=M*0;
for cc=1:length(S_set)
    S=S_set[cc];
    U=U_set[cc];
    Vh=Vh_set[cc];
    spin=SPIN_svd_set[cc]
    for cs=1:length(S)
        M_temp=M_temp+S[cs]*U[cs]*Vh[cs]*(spin[cs]*2+1);
    end
end
display(norm(M-M_temp)/norm(M))

S_set_combined,Vh_set_combined,U_set_combined,SPIN_svd_set_combined=combine_singlespin_sector(S_set,Vh_set,U_set,SPIN_svd_set,false)
M_temp=M*0;
for cc=1:length(S_set)
    S=S_set_combined[cc];
    U=U_set_combined[cc];
    Vh=Vh_set_combined[cc];
    spin=SPIN_svd_set_combined[cc]
    if S==[]
    else
        M_temp=M_temp+U*S*Vh*(spin*2+1);
    end
end
display(norm(M-M_temp)/norm(M))



#check GLR_svd
@tensor M[:]:=Ag'[-1,-3,1]*Ag[-2,-4,1];
M=permute(M,(1,2,),(3,4,));
for cc=1:pow-1
    M=M*M;
end
M_temp=M*0;
for cc=1:length(s_set)
    s=s_set[cc];
    u=u_set[cc];
    vh=vh_set[cc];
    spin=spin_svd_set[cc]
    for cs=1:length(s)
        M_temp=M_temp+s[cs]*u[cs]*vh[cs]*(spin[cs]*2+1);
    end
end
display(norm(M-M_temp)/norm(M))

s_set_combined,vh_set_combined,u_set_combined,spin_svd_set_combined=combine_singlespin_sector(s_set,vh_set,u_set,spin_svd_set,false)


M_temp=M*0;
for cc=1:length(s_set)
    s=s_set_combined[cc];
    vh=vh_set_combined[cc];
    u=u_set_combined[cc];
    spin=spin_svd_set_combined[cc]
    if s==[]
    else
        M_temp=M_temp+u*s*vh*(spin*2+1);
    end
end
display(norm(M-M_temp)/norm(M))


4.759743995378707e-15

4.796837186461085e-15

2.495328129374576e-10

2.4953281930164115e-10

In [198]:
#check FLR_eig
@tensor M[:]:=Ag'[-1,-4,1]*OO[-2,2,-5,1]*Ag[-3,-6,2];
M=permute(M,(1,2,3,),(4,5,6,));
for cc=1:pow-1
    M=M*M;
end
M_temp=M*0;
for cc=1:length(euL_set)
    euL=euL_set[cc];
    evR=evR_set[cc];
    evL=evL_set[cc];
    spin=SPIN_eig_set[cc]
    for cs=1:length(euL)
        @tensor coe=evR[cs][2,3,4,1]*evL[cs][1,2,3,4];
        M_temp=M_temp+euL[cs]^(pow)*evR[cs]*evL[cs]*(spin[cs]*2+1)/coe;
    end
end
display(norm(M-M_temp)/norm(M))

euR_set_combined,evL_set_combined,evR_set_combined,SPIN_eig_set_combined=combine_singlespin_sector(euR_set,evL_set,evR_set,SPIN_eig_set,true)
M_temp=M*0;
for cc=1:length(euR_set)
    euR=euR_set_combined[cc];
    evR=evR_set_combined[cc];
    evL=evL_set_combined[cc];
    spin=SPIN_eig_set_combined[cc]
    if euR==[]
    else
        for cc=1:pow-1
            euR=euR*euR;
        end
        M_temp=M_temp+evR*euR*evL*(spin*2+1);
    end
end
display(norm(M-M_temp)/norm(M))




#check GLR_eig
@tensor M[:]:=Ag'[-1,-3,1]*Ag[-2,-4,1];
M=permute(M,(1,2,),(3,4,));
for cc=1:pow-1
    M=M*M;
end
eur_set_combined,evl_set_combined,evr_set_combined,spin_eig_set_combined=combine_singlespin_sector(eur_set,evl_set,evr_set,spin_eig_set,true)

M_temp=M*0;
for cc=1:length(eul_set)
    eur=eur_set_combined[cc];
    evl=evl_set_combined[cc];
    evr=evr_set_combined[cc];
    spin=spin_eig_set_combined[cc]
    if eur==[]
    else
        for cc=1:pow-1
            eur=eur*eur;
        end
        M_temp=M_temp+evr*eur*evl*(spin*2+1);
    end
end
display(norm(M-M_temp)/norm(M))


1.1366317200174338e-13

1.2032371070539383e-14

3.8175291860148905e-15